In [1]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm


In [3]:

# Define transformations including random rotations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip images
    transforms.RandomRotation(10),  # Randomly rotate images by up to 10 degrees
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets
train_data_dir = '/content/drive/MyDrive/Robo/dataset/train/'
test_data_dir = '/content/drive/MyDrive/Robo/dataset/test/'

train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Load model
model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(dataloader, desc=f"Epoch [{epoch+1}/{num_epochs}]", unit="batch"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / total
        epoch_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

# Save the model
torch.save(model.state_dict(), 'freshness_detection_model.pth')

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test dataset: {accuracy:.2f}%')

# Evaluate the model
evaluate_model(model, test_loader)


Epoch [1/10]: 100%|██████████| 682/682 [34:50<00:00,  3.06s/batch]


Epoch [1/10], Loss: 0.0227, Accuracy: 87.78%


Epoch [2/10]: 100%|██████████| 682/682 [03:39<00:00,  3.10batch/s]


Epoch [2/10], Loss: 0.0116, Accuracy: 93.69%


Epoch [3/10]: 100%|██████████| 682/682 [03:37<00:00,  3.13batch/s]


...

Epoch [8/10], Loss: 0.0041, Accuracy: 97.91%


Epoch [9/10]: 100%|██████████| 682/682 [03:40<00:00,  3.09batch/s]


Epoch [9/10], Loss: 0.0028, Accuracy: 98.47%


Epoch [10/10]: 100%|██████████| 682/682 [03:37<00:00,  3.14batch/s]


Epoch [10/10], Loss: 0.0035, Accuracy: 98.34%
Accuracy on test dataset: 99.63%
